In [28]:
import torch
masked_state_dict = torch.load('masked_gpt2.pt')
N=100

In [29]:
from transformer_lens.HookedTransformer import HookedTransformer
gpt2_masked = HookedTransformer.from_pretrained(is_masked=True, model_name='gpt2')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [30]:
from transformer_lens.ioi_dataset import IOIDataset

ioi_dataset = IOIDataset(prompt_type="ABBA", N=N, nb_templates=1)
train_data = ioi_dataset.toks.long()


In [31]:
def logit_diff_from_ioi_dataset(
    logits: torch.Tensor, tokens: torch.Tensor, mean=False,
):
    assert tokens.shape == (
        N,
        16,
    ), tokens.shape  # TODO check this is not breaking things...
    assert len(logits.shape) == 3, logits.shape

    io_labels = tokens[:, 2]
    s_labels = tokens[:, 4]

    io_logits = logits[torch.arange(N), -2, io_labels]
    s_logits = logits[torch.arange(N), -2, s_labels]

    logit_diff = io_logits - s_logits
    if mean:
        return logit_diff.mean()
    else:
        return logit_diff


In [32]:
gpt2_masked.load_state_dict(masked_state_dict)
logit_diff_from_ioi_dataset(gpt2_masked(train_data), train_data, mean=True)

blocks.0.attn.hook_q
tensor(390.1159, device='cuda:0', grad_fn=<NormBackward1>)
blocks.0.attn.hook_k
tensor(648.3994, device='cuda:0', grad_fn=<NormBackward1>)
blocks.0.attn.hook_v
tensor(240.3886, device='cuda:0', grad_fn=<NormBackward1>)
blocks.1.attn.hook_q
tensor(365.2684, device='cuda:0', grad_fn=<NormBackward1>)
blocks.1.attn.hook_k
tensor(505.9655, device='cuda:0', grad_fn=<NormBackward1>)
blocks.1.attn.hook_v
tensor(230.0281, device='cuda:0', grad_fn=<NormBackward1>)
blocks.2.attn.hook_q
tensor(345.4847, device='cuda:0', grad_fn=<NormBackward1>)
blocks.2.attn.hook_k
tensor(695.6668, device='cuda:0', grad_fn=<NormBackward1>)
blocks.2.attn.hook_v
tensor(511.3076, device='cuda:0', grad_fn=<NormBackward1>)
blocks.3.attn.hook_q
tensor(4.8455, device='cuda:0', grad_fn=<NormBackward1>)
blocks.3.attn.hook_k
tensor(0., device='cuda:0', grad_fn=<NormBackward1>)
blocks.3.attn.hook_v
tensor(324.0270, device='cuda:0', grad_fn=<NormBackward1>)
blocks.4.attn.hook_q
tensor(537.7574, device='cu

tensor(20.3451, device='cuda:0', grad_fn=<MeanBackward0>)

In [33]:
# import numpy as np
# mask_dict = np.load('mask_dict.npy', allow_pickle=True).item()
# mask_dict["0.0.q"]

In [34]:
from transformer_lens import utils

# def make_forward_hooks(nodes_to_mask):
#     forward_hooks = []
#     for layer in range(12):
#         for head in range(12):
#             for qkv in ["q", "k", "v"]:
#                 mask_value = nodes_to_mask[f"{layer}.{head}.{qkv}"]
#                 def head_ablation_hook(value, hook):
#                     # print(f"Shape of the value tensor: {value.shape}")
#                     value[:, :, layer, :] *= mask_value
#                     return value

#                 a_hook = (utils.get_act_name(qkv, int(head)), head_ablation_hook)
#                 forward_hooks.append(a_hook)
#     return forward_hooks

from functools import partial 

def make_forward_hooks(nodes_to_mask):
    forward_hooks = []
    for layer in range(12):
        for head in range(12):
            for qkv in ["q", "k", "v"]:
                mask_value = nodes_to_mask[f"{layer}.{head}.{qkv}"]
                def head_ablation_hook(value, hook, head_idx, layer_idx, qkv_val, mask_value):
                    # print(f"Shape of the value tensor: {value.shape}")
                    # print(f"{layer}.{head}.{qkv}")
                    value[:, :, head_idx, :] *= mask_value
                    print(qkv_val, layer_idx, head_idx, torch.norm(value))
                    print("transformer lens mask values", mask_value)
                    return value

                a_hook = (utils.get_act_name(qkv, int(layer)), partial(head_ablation_hook, head_idx=head, layer_idx=layer, qkv_val=qkv, mask_value=mask_value))
                forward_hooks.append(a_hook)
    return forward_hooks


In [35]:
# new_gpt2 = HookedTransformer.from_pretrained(is_masked=False, model_name='gpt2')
# print(logit_diff_from_ioi_dataset(new_gpt2(train_data), train_data, mean=True))
new_gpt2 = HookedTransformer.from_pretrained(is_masked=False, model_name='gpt2')

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [36]:
mask_value_dict = {}
for layer_index, layer in enumerate(gpt2_masked.blocks):
    for head_index in range(12):
        for q_k_v in ["q", "k", "v"]:
            # total_nodes += 1
            if q_k_v == "q":
                mask_value = (
                    layer.attn.hook_q.sample_mask()[head_index].cpu().item()
                )
            if q_k_v == "k":
                mask_value = (
                    layer.attn.hook_k.sample_mask()[head_index].cpu().item()
                )
            if q_k_v == "v":
                mask_value = (
                    layer.attn.hook_v.sample_mask()[head_index].cpu().item()
                )
            mask_value_dict[f"{layer_index}.{head_index}.{q_k_v}"] = mask_value

mask_value_dict.values()
forward_hooks = make_forward_hooks(mask_value_dict)
logit_diff_from_ioi_dataset(new_gpt2.run_with_hooks(train_data, return_type="logits", fwd_hooks=forward_hooks), train_data, mean=True)


q 0 0 tensor(1025.2627, device='cuda:0', grad_fn=<NormBackward1>)
q 0 1 tensor(940.3607, device='cuda:0', grad_fn=<NormBackward1>)
q 0 2 tensor(907.1318, device='cuda:0', grad_fn=<NormBackward1>)
q 0 3 tensor(820.8910, device='cuda:0', grad_fn=<NormBackward1>)
q 0 4 tensor(751.3130, device='cuda:0', grad_fn=<NormBackward1>)
q 0 5 tensor(641.5286, device='cuda:0', grad_fn=<NormBackward1>)
q 0 6 tensor(591.0660, device='cuda:0', grad_fn=<NormBackward1>)
q 0 7 tensor(504.7794, device='cuda:0', grad_fn=<NormBackward1>)
q 0 8 tensor(410.9904, device='cuda:0', grad_fn=<NormBackward1>)
q 0 9 tensor(340.8100, device='cuda:0', grad_fn=<NormBackward1>)
q 0 10 tensor(241.9045, device='cuda:0', grad_fn=<NormBackward1>)
q 0 11 tensor(0., device='cuda:0', grad_fn=<NormBackward1>)
k 0 0 tensor(1572.4468, device='cuda:0', grad_fn=<NormBackward1>)
k 0 1 tensor(1506.3849, device='cuda:0', grad_fn=<NormBackward1>)
k 0 2 tensor(1450.0033, device='cuda:0', grad_fn=<NormBackward1>)
k 0 3 tensor(1186.5457, d

tensor(-0.0638, device='cuda:0', grad_fn=<MeanBackward0>)

In [37]:
forward_hooks[0]

('blocks.0.attn.hook_q',
 functools.partial(<function make_forward_hooks.<locals>.head_ablation_hook at 0x7f0a51c0beb0>, head_idx=0, layer_idx=0, qkv_val='q'))